In [1]:
# Bibliotecas
import tweepy
import pandas as pd
import datetime as dt
import yfinance as yf
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# Função para coletar os tweets e os exportar para arquivo Tweets.csv
def Coletar_Tweets():
    # Criar variável para dia anterior
    data = dt.datetime.now()
    data = data.replace(hour=0, minute=0, second=0, microsecond=0)
    data = data - dt.timedelta(days=1)

    # Criar variável com os termos a serem pesquisados
    Termos = "Bitcoin"

    # Obter Tokens
    with open("Twitter_API.txt", "r", encoding="utf-8") as f:
        Tokens = f.readlines()
        f.close()

    print("Token Adquirido")

    # Inicializar objeto Client para pesquisa de Tweets
    Cliente = tweepy.Client(
        bearer_token=Tokens[-1], access_token=Tokens[1], access_token_secret=Tokens[4])

    # Criar dicionário cujas keys são as horas do dia
    resultados = {}

    # Criar listas para segurar os Tweets e as datas correspondentes
    Tweets = []
    Datas = []

    # Recuperar 100 tweets para cada hora do dia anterior e preencher o dicionário
    for hora in range(0, 24):
        resultados[hora] = Cliente.search_recent_tweets(Termos, max_results=100, tweet_fields="created_at",
                                                        start_time=data, end_time=data.replace(hour=data.hour+1))
        for i in range(len(resultados[hora].data)):
            Datas.append(resultados[hora].data[i].created_at)
            Tweets.append(resultados[hora].data[i].text)
        data = data.replace(hour=hora)

    print("Tweets Processados")

    # Criar DataFrame Dados
    Df = pd.DataFrame()
    Df["Tweets"] = Tweets
    Df.index = Datas
    Df.sort_index(inplace=True)
    # Ler Arquivo existente
    Df_Antigo = pd.read_csv("Data/Tweets.csv", sep=";", index_col="Unnamed: 0")

    # Adicionar Novas Linhas

    Df = pd.concat([Df, Df_Antigo])

    # Exportar DataFrame para CSV.
    Df.to_csv("Data/Tweets.csv", sep=";")
    print("Dados exportados com Sucesso")

In [3]:
Coletar_Tweets()

Token Adquirido
Tweets Processados
Dados exportados com Sucesso


In [4]:
# Função para coletar os preços de BITCOIN/USD e exportar paraa Precos.csv
def Coletar_Precos():
    #Coletar Arquivo de Dados
    df = pd.read_csv("Data/Tweets.csv", sep=";", index_col="Unnamed: 0")
    df.sort_index(inplace=True)

    # Coletar a data do primeiro tweet coletado
    inicio = df.index[0]
    inicio = inicio[:10]

    # Coletar data do último tweet coletado
    fim = df.index[-1]
    fim = fim[:10]

    print("Horizonte temporal adquirido")

    # Obter preços do BTC-USD em intervalos de uma hora desde o primeiro (Somente Preço em que a hora fecha)
    Precos = yf.Ticker("BTC-USD").history(start=inicio, end=fim, interval="1h")["Close"]

    # Precos["Retornos"] = Precos["Close"].pct_change()

    # Exportar Preços
    Precos.to_csv("Data/Precos.csv", sep=";")
    print("Dados exportados com sucesso")


In [5]:
Coletar_Precos()

Horizonte temporal adquirido
Dados exportados com sucesso


In [6]:
# Analisar o sentimento de cada tweet coletado e exportar para Sentimento_Agregado.csv
def Sentimento():

    VADER = SentimentIntensityAnalyzer()

    # Ler Tweets Coletados
    Tweets = pd.read_csv("Data/Tweets.csv", sep=";", index_col="Unnamed: 0")

    # Lista para conter os sentimentos agregados
    Sentiment = []

    # Popular Lista
    for i in range(Tweets.shape[0]):
        Sentiment.append(VADER.polarity_scores(text=Tweets.iloc[i,0])["compound"])

    print("Sentimentos adquiridos")
    # Criar Coluna de sentimento no dataframe principal
    Tweets["Sentimento"] = Sentiment

    # Salvar Dataframe com sentiments
    Tweets.to_csv("Data/Sentimento_Agregado.csv", sep=";")

    print("Dados exportados com Sucesso")

In [7]:
Sentimento()

Sentimentos adquiridos
Dados exportados com Sucesso


In [8]:
# Juntar o sentimento agregado, tweet original e preço que o BITCOIN fecha na hora para uma único arquivo Base_Completa.csv
def Juntar_Base():
    
    # Ler dados de sentimentos e precos
    Precos = pd.read_csv("Data/Precos.csv", sep=";", index_col="Unnamed: 0")
    Sentimentos = pd.read_csv("Data/Sentimento_Agregado.csv", sep=";")
    
    # Converter coluna de horário para datetime, para podermos manipular os valores a serem compatíveis com aqueles
    # presentes no dataframe Precos, nos quais minutos e segundos são sempre 0
    Sentimentos["Unnamed: 0"] = pd.to_datetime(Sentimentos["Unnamed: 0"])
    
    # Trocar minutos diferentes de  0 por 0
    Sentimentos["Unnamed: 0"] = Sentimentos["Unnamed: 0"].mask(Sentimentos["Unnamed: 0"].dt.minute != 0, Sentimentos["Unnamed: 0"] \
    + pd.offsets.DateOffset(minute=0))    

    # Trocar segundos diferentes de 0 por 0
    Sentimentos["Unnamed: 0"] = Sentimentos["Unnamed: 0"].mask(Sentimentos["Unnamed: 0"].dt.second != 0, Sentimentos["Unnamed: 0"] \
    + pd.offsets.DateOffset(second=0))
    
    # Transformar coluna de datas em índice e ordená-la
    Sentimentos.set_index("Unnamed: 0", inplace=True)
    Sentimentos.sort_index(inplace=True)

    # Transformar índice do dataframe de preços em DateTime, para merge funcionar
    Precos.index = pd.to_datetime(Precos.index)

    # Juntar ambos os DataFrames
    Base_Completa = Sentimentos.merge(Precos, left_index=True, right_index=True)
    
    # Nomear Índice
    Base_Completa.index.name = "Horario"

    # Exportar para csv
    Base_Completa.to_csv("Data/Base_Completa.csv", sep=";")    

    print("Dados exportados com Sucesso")


In [9]:
Juntar_Base()

/home/coelho/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:760: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  warnings.warn(


Dados exportados com Sucesso
